In [1]:
%%capture
# Normally using pip install unsloth is enough

# Temporarily as of Jan 31st 2025, Colab has some issues with Pytorch
# Using pip install unsloth will take 3 minutes, whilst the below takes <1 minute:
pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
pip install --no-deps cut_cross_entropy unsloth_zoo
pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
pip install --no-deps unsloth

SyntaxError: invalid syntax (1392548529.py, line 5)

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/jkwade/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


Add LoRA adapters so we only need to update 1 to 10% of all parameters! This techniques allow us to fine-tune large models efficiently by adjusting only a subset of parameters instead of the entire model, reducing computational cost and memory usage.

In [ ]:

# Use PEFT (Parameter-Efficient Fine-Tunning) on our selected model 
model = FastLanguageModel.get_peft_model(
    model,
    # rank of the LoRA matrix. The rank determines the number of parameters added for adaptation.
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128

    # which parts (modules) of the model to apply LoRA fine-tuning.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],

    lora_alpha = 16, #how much influence the fine-tuned weights have on the original model.

    #Apply dropout to LoRA layers during fine-tuning to prevent overfitting
    lora_dropout = 0, # Supports any, but = 0 is optimized

    bias = "none",    # Supports any, but = "none" is optimized

    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context

    random_state = 3407, #seed for reproducibility, ensuring that results remain consistent across runs.

    # Rank-Stabilized LoRA (RS-LoRA), which helps stabilize training when fine-tuning models with very high rank 
    use_rslora = False,  #  disable support rank stabilized LoRA (default)

    loftq_config = None, # And LoftQ
)

DATA PROCESSING Section

In [4]:
import json
from datasets import Dataset

# Read the JSONL file
filepath = "/home/jkwade/Code/CareConnect/processed_data/symptoms_diagnosis_data/disease_sympts_prec_full-2.jsonl"
dataset = []
with open(filepath, "r", encoding="utf-8") as file:
    for line in file:
        dataset.append(json.loads(line.strip()))

def convert_to_conversation(sample):
    conversation = [
        {
            "from": "system",
            "value": sample['instruction']
        },
        {
            "from": "human",
            "value": "Predict the disease based on these symptoms: " + sample['input']
        },
        {
            "from": "gpt",
            "value": sample['output']
        }
    ]
    return {"conversations": conversation}

# Fixed the function name in the list comprehension
data_temp = [convert_to_conversation(data) for data in dataset]

# The structure needs to be modified since Dataset.from_dict expects a different format
data_dict = {
    "conversations": [item["conversations"] for item in data_temp]
}

# Added missing closing parenthesis
dataset = Dataset.from_dict(data_dict)

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset(dataset, split = "train")

In [ ]:
print(dataset[:5])


In [ ]:
from unsloth.chat_templates import standardize_sharegpt

dataset = standardize_sharegpt(dataset) #Ensures the dataset is in a clean and consistent format
dataset = dataset.map(formatting_prompts_func, batched = True,) #Converts the standardized data into the final format expected by the LLaMA 3.2 chat template.

In [ ]:
dataset[5] ["conversations"] #  raw data for debugging or validation.

In [ ]:
dataset[5]["text"] # verify that the formatting and preprocessing worked as expected.

Training setup


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,  #model to finetune  (were using llama3.2)
    tokenizer = tokenizer, #selects our toenizer from the code block above
    train_dataset = dataset, #basially all this stuff in predefined in the blocks above just ctl+click 
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(# alright now it gets freaky 
        per_device_train_batch_size = 2,    #each GPU processes two examples per bactch
        gradient_accumulation_steps = 4,    #  Simulates a batch size of 2 × 4 = 8 by accumulating gradients over 4 steps before updating weights.
        warmup_steps = 5,   #this will gradually increse the learning rate after the first 5 steps 
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60, #only 5 steps of training 
        learning_rate = 2e-4, # initial learning rate.
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,  #logs training metrics  
        optim = "adamw_8bit",   #this is a optimizer with 8-bit weights. this is reduced for memory usage
        weight_decay = 0.01,    #this helps overfitting, applies a weight decay to regualize the model 
        lr_scheduler_type = "linear",
        seed = 3407,    #Sets the random seed for reproducibility
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Very masking is done correctly 

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

In [ ]:
#Show Computer memory stats 
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

More memory stats

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
import torch
torch.cuda.empty_cache()

Local Saving

In [ ]:
model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit",)


In [ ]:
!python llama.cpp/convert_hf_to_gguf.py merged_model --outfile text2sql_8.gguf --outtype q8_0


In [ ]:
!ollama create unsloth_model_8 -f Modelfile